## Manipulação e Tratamento de Dados - Camada Silver

Camada de dados tratada e normalizada, confíavel para uso.

In [1]:

# IMPORTS AND LIBRARIES
import os
import boto3
from botocore.exceptions import ClientError
from datetime import datetime
import logging

# Configuração do logger
logger = logging.getLogger("minio_logger")
logger.setLevel(logging.INFO)


# Configurando o formato do log
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
console_handler = logging.StreamHandler()
console_handler.setFormatter(formatter)
logger.addHandler(console_handler)


# PySpark Libraries
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.functions import col, explode, lit, from_json, sum as _sum, avg, expr, round


In [2]:
# Variáveis Globais e de Ambiente para o Projeto.

os.environ["MINIO_KEY"] = "developer"
os.environ["MINIO_SECRET"] = "developer01"
os.environ["MINIO_ENDPOINT"] = "http://minio:9000"


# Na ausência do dbtuils ou do Metastore vou usar o boto para me ajudar
# a iteragir com o storage
s3_client = boto3.client(
    's3',
    endpoint_url = os.environ.get("MINIO_ENDPOINT"),
    aws_access_key_id = os.environ.get("MINIO_KEY"),
    aws_secret_access_key = os.environ.get("MINIO_SECRET")
)

bucket_name = "bank-databr"

# Paths Data Storage
root_path_dir = f"{bucket_name}"
landing_path_dir = f"{root_path_dir}/landing/bacen"
bronze_path_dir = f"{root_path_dir}/bronze"
silver_path_dir = f"{root_path_dir}/silver"

# Partição da tabela Bronze, Data de referência
dt_partition = datetime.now().strftime("%Y-%m-%d")

In [3]:

spark = SparkSession.builder \
                    .appName("SilverLayer") \
                    .config("spark.hadoop.fs.s3a.endpoint", os.environ["MINIO_ENDPOINT"]) \
                    .config("spark.hadoop.fs.s3a.access.key", os.environ["MINIO_KEY"]) \
                    .config("spark.hadoop.fs.s3a.secret.key", os.environ["MINIO_SECRET"]) \
                    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
                    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
                    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
                    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
                    .getOrCreate()


/opt/spark/bin/load-spark-env.sh: line 68: ps: command not found


:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
software.amazon.awssdk#s3 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-cea39eac-65db-49df-ba45-b51eb37b9a35;1.0
	confs: [default]
	found software.amazon.awssdk#s3;2.26.30 in central
	found software.amazon.awssdk#aws-xml-protocol;2.26.30 in central
	found software.amazon.awssdk#aws-query-protocol;2.26.30 in central
	found software.amazon.awssdk#protocol-core;2.26.30 in central
	found software.amazon.awssdk#sdk-core;2.26.30 in central
	found software.amazon.awssdk#annotations;2.26.30 in central
	found software.amazon.awssdk#http-client-spi;2.26.30 in central
	found software.amazon.awssdk#utils;2.26.30 in central
	found org.reactivestreams#reactive-streams;1.0.4 in central
	found org.slf4j#slf4j-api;1.7.36 in central
	found software.amazon.awssdk#metric

### Funções Utilitárias - Utils

In [5]:

def check_table_exists_in_metastore(schema_name:str, table_name: str) -> bool:
    """ Verifica se a tabela já existe no schema indicado dentro do Catalógo """

    return spark._jsparkSession.catalog() \
                               .tableExists(f"{schema_name}.{table_name}")


def check_table_exists_by_location(s3_minio_client:boto3.client, destionation_table_path: str) -> bool:
    """ Verifica se a tabela já existe, usando o gerenciador do storage """

    # Vou adaptar usando o boto3 já que não tenho por aqui nem o dbutils nem um metastore para checkar via tableExists()
    
    # Extrai o bucket e o prefixo (path)
    bucket_name, path = destionation_table_path.replace("s3a://", "").split("/", 1)
    
    try:
        # Verifica se o diretório existe no S3 (se o prefixo da pasta tiver objetos)
        response = s3_minio_client.list_objects_v2(Bucket=bucket_name, Prefix=path, MaxKeys=1)
        return 'Contents' in response
        
    except ClientError as e:
        return False


def write_delta_table_by_location(df_input: DataFrame, destionation_table_path: str) -> None:
    """ Escreve a tabela Delta no Storage de forma particionada
        Se a tabela já existir realiza upsert na partição indicada
        Caso não exista primeiramente cria a tabela no modo 'append'

        Args:
            df_input: dataframe a ser gravado no storage
            destionation_table_path: local de destino no storage
    """

    if check_table_exists_by_location(s3_client, destionation_table_path): 

        print(f"Tabela já existe: escrevendo nova partição em:\n\t*{destionation_table_path}")

        df_input.write \
                .format("delta") \
                .mode("append") \
                .option("replaceWhere", f"partition = {dt_partition}") \
                .partitionBy("dt_partition") \
                .save(destionation_table_path)   
    else:
        
        print(f"Tabela ainda não existe, criando nova tabela em: \n\t*{destionation_table_path}")

        #TODO: atualizar para considerar o EvolutionSchema, option("mergeSchema": True)
        # No formato atual estou considerando somente como EnforceSchema.

        df_input.write \
                .format("delta") \
                .mode("append") \
                .partitionBy("dt_partition") \
                .save(destionation_table_path) 

   

### 01. Pagamentos  Trimestrais


Leitura e tratamento da tabela Bronze. Aqui vamos aplicar algumas transformações e tratamentos também vamos definir um schema final desejado para a tabela na silva.

In [6]:
# Lendo tabela de Pagamentos Trimestrais da Bronze

table_name = "b_pagamentos_trimestrais_bc"
dt_ref_carga= "2025-01-20"
# Fixando a partição a ser lida. Caso fosse fosse algo produtivo usando um orquestrador como Control-M ou Airflow
# este valor seria uma variável de ambiente.

data_source_file_path = f"s3a://{bronze_path_dir}/{table_name}"
print(f"* Data Source Pagamentos Trimestrais: {data_source_file_path}")


df_pagamentos_trimestral_bronze = spark.read \
                                       .format("delta") \
                                       .load(data_source_file_path) \
                                       .where(col('dt_partition') == dt_ref_carga)


print("\n* Schema Original do arquivo origem")
df_pagamentos_trimestral_bronze.printSchema
df_pagamentos_trimestral_bronze.show(n=1, vertical=True, truncate=True)


* Data Source Pagamentos Trimestrais: s3a://bank-databr/bronze/b_pagamentos_trimestrais_bc


25/01/31 23:34:37 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.



* Schema Original do arquivo origem


-RECORD 0--------------------------------------
 @odata.context         | https://was-p.bcn... 
 value                  | [{2024-09-30, 152... 
 file_path              | s3a://bank-databr... 
 file_name              | data_18_01_2025_1... 
 file_size              | 19142                
 file_block_start       | 0                    
 file_block_length      | 19142                
 file_modification_time | 2025-01-18 20:40:39  
 ingestion_engine       | python_dlt           
 dt_partition           | 2025-01-20           
only showing top 1 row



#### Aplicando Transformações e Schema Final

In [12]:
# Vamos transformar a tabela original, explodindo os campos JSON/Struct para novas colunas e filtrando algumas variáveis

# Explodindo o valor da coluna "value" em múltiplas colunas
df_pagamentos_trimestral_transformed = df_pagamentos_trimestral_bronze.withColumn('value_struct', explode(col("value"))) \
                                                                      .select( col("value_struct.*") ) \
                                                                      .drop(*['@odata.context', 'value']) \
                                                                      .withColumn('dt_partition', lit(dt_ref_carga) )


# Vou criar duas novas colunas, uma contendo a Soma da quantidade de transações e outra contendo a soma dos valores de todas as transações.

colunas_quantidades = [
    column_name
    for column_name in df_pagamentos_trimestral_transformed.columns
    if column_name.startswith("quantidade")
]

colunas_valores = [
    column_name
    for column_name in df_pagamentos_trimestral_transformed.columns
    if column_name.startswith("valor")
]


df_pagamentos_trimestral_transformed = \
    df_pagamentos_trimestral_transformed.withColumn('TotalQtdTransacoes', round(expr('+'.join(colunas_quantidades)), 2) ) \
                                        .withColumn('TotalValores', round(expr('+'.join(colunas_valores)), 2) )

# Schema Original do arquivo origem
df_pagamentos_trimestral_transformed.printSchema()
df_pagamentos_trimestral_transformed.show(n=1, vertical=True, truncate=True)


root
 |-- datatrimestre: string (nullable = true)
 |-- quantidadeBoleto: double (nullable = true)
 |-- quantidadeCartaoCredito: double (nullable = true)
 |-- quantidadeCartaoDebito: double (nullable = true)
 |-- quantidadeCartaoPrePago: double (nullable = true)
 |-- quantidadeCheque: double (nullable = true)
 |-- quantidadeConvenios: double (nullable = true)
 |-- quantidadeDOC: double (nullable = true)
 |-- quantidadeDebitoDireto: double (nullable = true)
 |-- quantidadePix: double (nullable = true)
 |-- quantidadeSaques: double (nullable = true)
 |-- quantidadeTEC: double (nullable = true)
 |-- quantidadeTED: double (nullable = true)
 |-- quantidadeTransIntrabancaria: double (nullable = true)
 |-- valorBoleto: double (nullable = true)
 |-- valorCartaoCredito: double (nullable = true)
 |-- valorCartaoDebito: double (nullable = true)
 |-- valorCartaoPrePago: double (nullable = true)
 |-- valorCheque: double (nullable = true)
 |-- valorConvenios: double (nullable = true)
 |-- valorDOC: d

-RECORD 0-------------------------------------
 datatrimestre                | 2024-09-30    
 quantidadeBoleto             | 1525449.9     
 quantidadeCartaoCredito      | 5061154.29    
 quantidadeCartaoDebito       | 4121806.1     
 quantidadeCartaoPrePago      | 3132432.21    
 quantidadeCheque             | 44243.73      
 quantidadeConvenios          | 661708.42     
 quantidadeDOC                | 0.0           
 quantidadeDebitoDireto       | 4056152.31    
 quantidadePix                | 1.654642727E7 
 quantidadeSaques             | 664224.81     
 quantidadeTEC                | 0.0           
 quantidadeTED                | 203394.26     
 quantidadeTransIntrabancaria | 771817.52     
 valorBoleto                  | 2463425.06    
 valorCartaoCredito           | 662702.46     
 valorCartaoDebito            | 241932.98     
 valorCartaoPrePago           | 78531.64      
 valorCheque                  | 194330.6      
 valorConvenios               | 961282.82     
 valorDOC    

#### Sumário - Algumas estatísticas sobre cada variável no DataSet

In [9]:
df = df_pagamentos_trimestral_transformed

# Filtra somente colunas númericas
df_numerics = df.select([
    col(column_name)
    for column_name, dtypes in df.dtypes
    if dtypes in ["double", "int"]
])

# Recupera Sumário de Métricas do DataFrame
resumo_estatistico = df_numerics.summary().toPandas().T

resumo_estatistico.reset_index(inplace=True)

# Renomeia colunas com o conteúdo da primeira linha
resumo_estatistico.columns = resumo_estatistico.iloc[0]
# Deleta primeira linha após usar como cobeçalho
resumo_estatistico = resumo_estatistico.drop(index=0).reset_index(drop=True)
resumo_estatistico.rename(columns={"summary":"ColumnName"}, inplace=True)

resumo_estatistico


,ColumnName,count,mean,stddev,min,25%,50%,75%,max
0,quantidadeBoleto,161,1389390.3082608692,154697.1615259209,1020583.81,1256812.05,1452479.44,1497704.81,1538367.11
1,quantidadeCartaoCredito,161,3489635.5491304337,1015743.6959186543,1907929.12,2516328.04,3679415.97,4434832.09,5061154.29
2,quantidadeCartaoDebito,161,3447517.5821739123,596469.1752884315,2259584.47,2957015.55,3589968.33,3952365.16,4238653.32
3,quantidadeCartaoPrePago,161,1543923.8791304342,1006682.746567023,58847.58,577500.51,1507246.16,2426618.73,3132432.21
4,quantidadeCheque,161,81366.07565217391,30667.439926231105,44243.73,56339.37,75417.98,91594.82,139621.22
5,quantidadeConvenios,161,763740.3582608697,59051.488967287,661708.42,733937.43,766141.92,786763.01,913784.42
6,quantidadeDOC,161,28485.392608695664,23228.71148906417,0.0,9871.74,18047.28,51756.51,74377.91
7,quantidadeDebitoDireto,161,2120465.562608695,725863.1564419273,1381466.73,1542581.41,1809992.06,2557306.25,4056152.31
8,quantidadePix,161,5252058.1191304345,5478084.669775289,0.0,0.0,3836285.6,9396579.03,1.654642727E7
9,quantidadeSaques,161,891650.0278260867,169594.27000844522,655963.27,725040.4,918141.51,1034924.99,1237366.49


#### Data Quality

Vamos verificar o schema inferido validando os tipos dos dados e valores carregados



In [10]:

from pyspark.sql.functions import count as _count, when

def resumo_estatistico(df: DataFrame) -> DataFrame:
    """ Gera um resumo com metadados e estatíticas básicas sobre o DataFrame """

    # Recuperando o Nome e tipos de cada Coluna
    column_types = df.dtypes

    # Recupera a quantidade de valores Nulos/Empty/Missing para cada coluna

    null_counts = df.select([
                                _count(
                                    when(col(column_name).isNull(), column_name)
                                ).alias(column_name)
                                for column_name in df.columns
                            ]).collect()[0]

    
    # Montando o resumo final
    resumo = [
        (column_name, dtype, null_counts[column_name])
        for column_name, dtype in column_types
    ]

    # Dataframe resposta
    schema = ["ColumnName", "DataType", "CountMissings"]

    return spark.createDataFrame(data = resumo, schema = schema)




df_statistics = resumo_estatistico(df_pagamentos_trimestral_transformed)

df_statistics.show(truncate=False, n=30)

+----------------------------+--------+-------------+
|ColumnName                  |DataType|CountMissings|
+----------------------------+--------+-------------+
|datatrimestre               |string  |0            |
|quantidadeBoleto            |double  |0            |
|quantidadeCartaoCredito     |double  |0            |
|quantidadeCartaoDebito      |double  |0            |
|quantidadeCartaoPrePago     |double  |0            |
|quantidadeCheque            |double  |0            |
|quantidadeConvenios         |double  |0            |
|quantidadeDOC               |double  |0            |
|quantidadeDebitoDireto      |double  |0            |
|quantidadePix               |double  |0            |
|quantidadeSaques            |double  |0            |
|quantidadeTEC               |double  |0            |
|quantidadeTED               |double  |0            |
|quantidadeTransIntrabancaria|double  |0            |
|valorBoleto                 |double  |0            |
|valorCartaoCredito         

#### Gravando a tabela final na Silver

In [11]:
print(" Gravando Dados na Silver")

schema_name = "db_bank_databr"
table_name = "s_cartoes_trimestral_bc"


destionation_table_path = f"s3a://{silver_path_dir}/{table_name}"

write_delta_table_by_location(df_pagamentos_trimestral_transformed, destionation_table_path)

 Gravando Dados na Silver
Tabela ainda não existe, criando nova tabela em: 
	*s3a://bank-databr/silver/s_cartoes_trimestral_bc
